In [1]:
import tensorflow as tf
from tensorflow.keras import Input, layers, Model, losses, optimizers
import tensorflow_probability as tfp

In [4]:
import numpy as np

In [13]:
state = np.array([0, 0, 0])
state = np.reshape(state, (1, 3))
action = np.array([10]) 
y = np.array([-60000])

In [15]:
"""
https://stackoverflow.com/questions/51680818/keras-custom-loss-as-a-function-of-multiple-outputs/51685637
"""

inputs = Input(shape=(3,))
prevAction = Input(shape=(1,))
target = Input(shape=(1,))

mu = layers.Dense(3)(inputs)
std = 1.0


normal_dist = tfp.distributions.Normal(mu, std)
# normal distribution
# sampling
def sampling(args):
    return normal_dist.sample(1)

a = layers.Lambda(sampling, output_shape=(3,))([mu, std])
# clipping
# a = tf.clip_by_value(a, env.action_space.low[0], env.action_space.high[0])

# consider training in a batch for 1-episode, speed up
outputs = a
model = Model(inputs=[inputs, prevAction, target], outputs=[outputs, mu])

loss = -normal_dist.log_prob(prevAction) * target
optimizer = "Adam"
model.add_loss(loss)
model.compile(optimizer=optimizer)

model.fit([state, action, y])

model.predict([state, action, y])

Train on 1 samples
1/1 [==============================] - 0s 140ms/sample - loss: -3055136.2500


[array([[[-1.2866951 , -0.44198593, -0.4905026 ]]], dtype=float32),
 array([[-0.001, -0.001, -0.001]], dtype=float32)]

In [16]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 3)            12          input_19[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_Identity_6 (TensorF [(None, 3)]          0           dense_6[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Shape_12 (TensorFlo [(2,)]               0           tf_op_layer_Identity_6[0][0]     
____________________________________________________________________________________________

([-1226.140364950204, -1226.140364950204, 1232.2206230353208],
 array([10., 10., 10.]))